In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from quickdraw import QuickDrawDataGroup
from libs import nb_utils, utils
from cppn import CPPN

print(tf.__version__)

1.13.1


In [2]:
#
# Note: Downloads a huge data set.
#
#
'''total_drawings = 121380
balloons = QuickDrawDataGroup("hot air balloon", max_drawings=total_drawings, refresh_data=True, print_messages =True)
for i in range(total_drawings):
    balloon = balloons.get_drawing(i)
    balloon.image.save("data/balloon/balloon_"+str(i)+".gif")'''


'total_drawings = 121380\nballoons = QuickDrawDataGroup("hot air balloon", max_drawings=total_drawings, refresh_data=True, print_messages =True)\nfor i in range(total_drawings):\n    balloon = balloons.get_drawing(i)\n    balloon.image.save("data/balloon/balloon_"+str(i)+".gif")'

In [21]:
tf.reset_default_graph()

batch_size=100
n_pixels=32
n_channels=1
input_shape=[None, n_pixels, n_pixels, n_channels]
z_dim=32

X = tf.placeholder(name='X', shape=input_shape, dtype=tf.float32)

In [22]:
#Used by the discriminator. Creates the latent vector
def encoder(x, channels=[50, 50, 50 ,50], filter_sizes=[4, 4, 4, 4], activation=tf.tanh, reuse=None):
    h=x
    hs=[]
    
    for layer_i in range(len(channels)):
        with tf.variable_scope('layer{}'.format(layer_i+1), reuse=reuse):
            h,W = utils.conv2d(x, n_output=channels[layer_i], k_h=filter_sizes[layer_i], k_w=filter_sizes[layer_i], reuse=reuse)
            h = activation(h)
            hs.append(h)
    return h, hs

def discriminator(x, channels=[50, 50, 50 ,50], filter_sizes=[4, 4, 4, 4], activation=utils.lrelu, reuse=None):
    h=None
    with tf.variable_scope('discriminator', reuse=reuse):
        h, hs = encoder(x, channels, filter_sizes, activation, reuse=reuse)
        shape = h.get_shape().as_list()
        h = tf.reshape(h, [-1, shape[1]*shape[2]*shape[3]])
        
        D,W = utils.linear(x, activation=tf.sigmoid, n_output=1, reuse=reuse)
    return D, h

In [23]:
D_real, z = discriminator(X)

In [24]:
def generator(z, x_y_dim=n_pixels, z_dim=32, channels=[128, 128, 128, n_channels], activation=tf.tanh, reuse=None):
    with tf.variable_scope('generator'):
        g = CPPN(x_y_dim, x_y_dim, n_channels, channels[0], z_dim)
        [t_x, t_y, t_r, t_z] = g.create_inputs()
        G = g.generate(t_x, t_y, t_r, t_z)
    return G
    

In [25]:
#Get the first sample
G = generator(z, z_dim=z_dim, reuse=True)

In [26]:
D_fake, z_fake=discriminator(G, reuse=True)
D_fake

<tf.Tensor 'discriminator_1/fc/Sigmoid:0' shape=(1, 1) dtype=float32>

In [27]:
with tf.variable_scope('loss/generator'):
    loss_G = tf.reduce_mean(utils.binary_cross_entropy(D_fake, tf.ones_like(D_fake)))
with tf.variable_scope('loss/discriminator/real'):
    loss_D_real = utils.binary_cross_entropy(D_real, tf.ones_like(D_real))
with tf.variable_scope('loss/discriminator/fake'):
    loss_D_fake = utils.binary_cross_entropy(D_fake, tf.ones_like(D_fake))
with tf.variable_scope('loss/discriminator'):
    loss_D = tf.reduce_mean((loss_D_real + loss_D_fake) / 2)

In [28]:
graph = tf.get_default_graph()
nb_utils.show_graph(graph.as_graph_def())

In [32]:
vars_d = [v for v in tf.trainable_variables() if v.name.startswith('discriminator')]
vars_g = [v for v in tf.trainable_variables() if v.name.startswith('generator')]

d_reg = tf.contrib.layers.apply_regularization(tf.contrib.layers.l2_regularizer(1e-6), vars_d)
g_reg = tf.contrib.layers.apply_regularization(tf.contrib.layers.l2_regularizer(1e-6), vars_g)

In [33]:
learning_rate = 0.0001

lr_g = tf.placeholder(tf.float32, shape=[], name='learning_rate_g')
lr_d = tf.placeholder(tf.float32, shape=[], name='learning_rate_d')

In [34]:
opt_g = tf.train.AdamOptimizer(learning_rate=lr_g).minimize(loss_G + g_reg, var_list=vars_g)
opt_d = tf.train.AdamOptimizer(learning_rate=lr_d).minimize(loss_D + d_reg, var_list=vars_d)